In [ ]:
# запускаем в colab или локально?
try:
    from google.colab import drive
    colab = True
except ImportError:
    colab = False

print(f"colab: {colab}")

In [ ]:
# установка необходимых пакетов в colab
if colab:
    !pip install rootutils -q
    !pip install torchinfo -q
    !pip install torchmetrics -q
    !pip install livelossplot -q

In [ ]:
# монтирование google диска и установка
# рабочей директории в `computer-vision`

import os
from rootutils import setup_root

if colab:
    drive.mount("/content/drive", force_remount=True)
    os.chdir("drive/MyDrive/computer-vision")
    root = setup_root(".", indicator="homeworks", pythonpath=True)
else:
    root = setup_root(".", indicator="homeworks", pythonpath=True)

os.chdir(root)
print(f"working directory: {os.getcwd()}")


In [ ]:
# создание директории для данных

from pathlib import Path

if colab:
    DATA_DIR = Path("/content/data")
else:
    DATA_DIR = root / "data"

DATA_DIR.mkdir(exist_ok=True)

print(f"DATA_DIR: {DATA_DIR}")

In [ ]:
# настройка matplotlib

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format="retina"

plt.style.use("seaborn-v0_8-notebook")

## **Полносвязные нейронные сети. Методы регуляризации**

### **1. Полносвязные нейронные сети**

#### **1.1 Пример полносвязной сети**

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/fcnn.jpg" width="600px"/>
    </figure>
</center>

- У этой сети один входной слой, один выходной слой и три скрытых слоя.

- Параметры нейронной сети
$$
    \mathbf{w} = \left\{
        \boldsymbol{\beta}_0,\boldsymbol{\Omega}_0,
        \boldsymbol{\beta}_1,\boldsymbol{\Omega}_1,
        \boldsymbol{\beta}_2,\boldsymbol{\Omega}_2,
        \boldsymbol{\beta}_3,\boldsymbol{\Omega}_3
    \right\}
$$

    Число параметров равно:
$$
    (D_1\cdot D_i + D_1) + (D_2\cdot D_1 + D_2) + (D_3\cdot D_2 + D_3) + (D_0\cdot D_3 + D_0) = 43
$$

- Вычисления слоев нейронной сети:
    $$
        \mathbf{h}_1 = \mathbf{a}\left[\boldsymbol{\beta}_0 + \mathbf{\Omega}_0 \mathbf{x}\right]
    $$
    $$
        \mathbf{h}_2 = \mathbf{a}\left[\boldsymbol{\beta}_1 + \mathbf{\Omega}_1 \mathbf{h}_1\right]
    $$
    $$
        \mathbf{h}_3 = \mathbf{a}\left[\boldsymbol{\beta}_2 + \mathbf{\Omega}_2 \mathbf{h}_2\right]
    $$
    $$
        \mathbf{y} = \boldsymbol{\beta}_3 + \mathbf{\Omega}_3 \mathbf{h}_3
    $$

    где $\mathbf{a}$ - функция активации, обычно это $\mathrm{ReLU}$ (Rectified Linear Unit):
    <center>
        <figure>
            <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/relu.png" width="200px"/>
        </figure>
    </center>

$$
    a[z] = \mathrm{ReLU}[z] =
    \begin{cases}
        0 & z < 0\\
        z & z \geqslant 0
    \end{cases}
$$

- Сеть называется **полносвязной**, поскольку каждый элемент (node) связан со всеми предыдущими элементами.


#### **1.2 Dropout**

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/dropout.jpg" width="500px"/>
    </figure>
</center>

- Для уменьшения переобучения после скрытых слоев иногда добавляют **dropout**.

- Dropout обнуляет скрытые элементы с вероятностью $p$ (обычно $p=50\%$) на каждой итерации SGD.

- Получается так, что на каждой итерации мы обучаем немного другую сеть.

- В PyTorch, в режиме обучения `model.train()`, выходы оставшихся после dropout скрытых элементов умножаются на
$$
    \frac{1}{1-p}
$$

- При валидации и тестировании, в режиме `model.eval()` скрытые элементы не обнуляются и их выходы не изменяются.

#### **1.3 Пример полносвязной сети в PyTorch**

In [ ]:
from torch import nn

class Net(nn.Module):
    def __init__(self, Di=3, D1=4, D2=2, D3=3, Do=2):
        super().__init__()

        act = nn.ReLU()

        self.layers = nn.Sequential(
            nn.Linear(Di, D1),
            act,
            nn.Dropout(0.5),
            nn.Linear(D1, D2),
            act,
            nn.Dropout(0.5),
            nn.Linear(D2, D3),
            act,
            nn.Dropout(0.5),
            nn.Linear(D3, Do),
        )

    def forward(self, x):
        y = self.layers(x)

        return y

Протестируем работу сети:

In [ ]:
import torch

# модель сети
model = Net()

# батч размера 5 из 3-х случайных чисел
x = torch.rand((5, 3))

# forward pass
y = model(x)

print(f"x.shape: {x.shape}")
print(f"\nx: {x}")
print(f"\ny.shape: {y.shape}")
print(f"\ny: {y}")

Выведем информацию об архитектуре сети:

In [ ]:
import torchinfo

print(torchinfo.summary(model, input_size=x.shape, device="cpu"))

### **2. Методы регуляризации**

- **Регуляризация в математике**: добавление к функции
    потерь заданных в явном виде членов, которые приводят к определенному
    выбору параметров.

- **Регуляризация в машинном обучении**: любые методы, которые улучшают обобщающую способность модели.

Наиболее распространенные методы регуляризации:

- Выбор архитектуры модели.

- Инициализация параметров модели.
- Выбор метода оптимизации для обучения сети.
- Регуляризация функции потерь.
- Аугментация изображений во время обучения.
- Learning rate scheduler.
- Ранняя остановка обучения (early stopping).

#### **2.1 Выбор архитектуры модели**

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/transfer.jpg" width="800px"/>
    </figure>
</center>

- **Neural architecture search (NAS)**: подбираем подходящую архитектуру вручную или с помощью специализированных фреймворков.

- **Transfer learning**:
    - берем готовую модель, обученную на вторичной задаче, удаляем последние слои и замещаем их новыми слоями, свойственными нашей задаче

    - обучаем только новые слои или обучаем всю сеть (**fine tuning**)


#### **2.2 Инициализация параметров модели**

##### **Проблема роста и убывания данных**
<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/gradient_init.jpg" width="500px"/>
    </figure>
</center>

- Прямой проход в слое $k$ полносвязной сети:
$$
    \mathbf{h}_{k+1} = \mathbf{a}\left[\mathbf{f}_{k+1}\right] =
    \mathbf{a}\left[\boldsymbol{\beta}_k + \boldsymbol{\Omega}_k \mathbf{h}_k\right]
$$
- Инициализируем параметры следующим образом:
$$
    \boldsymbol{\Omega}_k\sim N\left(0\,,\sigma^2_\Omega\right)
$$
- Проблема роста и убывания данных:
$$    
    \sigma^2_\Omega \gg 1
    \quad\Rightarrow\quad  
    \sigma^2_{h_k} \quad
    \text{- увеличивается при}
    \quad k=1,2,\ldots ,K
$$
$$
    \sigma^2_\Omega \ll 1
    \quad\Rightarrow\quad  
    \sigma^2_{h_k} \quad
    \text{- уменьшается при} \quad k=1,2,\ldots ,K
$$

##### **Проблема исчезающих и взрывных градиентов**
<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/gradient_init.jpg" width="500px"/>
    </figure>
</center>

- Обратный проход в слоях полносвязной сети:
$$
    \frac{\partial L}{\partial\mathbf{f}_{k-1}} =
    \mathbb{I}[\mathbf{f}_{k-1}>0]
    \odot
    \left(
        \boldsymbol{\Omega}^\top_k \frac{\partial L}{\partial\mathbf{f}_k}
    \right)
    \quad
    k=K,K-1,\ldots,1
$$
- Проблема исчезающих и взрывных градиентов:
$$
    \sigma^2_\Omega \gg 1
    \quad\Rightarrow\quad  
    \text{дисперсия}
    \quad \frac{\partial L}{\partial\mathbf{f}_k}\quad
    \text{увеличивается при} \quad k=K,K-1,\ldots ,1
$$
$$
    \sigma^2_\Omega \ll 1
    \quad\Rightarrow\quad  
    \text{дисперсия}\quad \frac{\partial L}{\partial\mathbf{f}_k}\quad
    \text{уменьшается при} \quad k=K,K-1,\ldots ,1
$$


##### **Инициализация Хе (Kaiming He)**
<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/layer.png" width="200px"/>
    </figure>
</center>

В случае функции активации ReLU:
$$
    a[x] = \mathrm{ReLU}[x]
$$
Инициализация Хе:
$$
    \boldsymbol{\Omega}_k\sim N\left(0\,,\sigma^2_\Omega\right)\,,
    \qquad
    \sigma_{\Omega}^2  = \frac{4}{D_{\mathrm{in}} + D_{\mathrm{out}}}
$$
Позволяет предотвратить взрыв и затухание данных и градиентов.

##### **Инициализация Ксавьера (Xavier Glorot)**
<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/layer.png" width="200px"/>
    </figure>
</center>

В случае нечетной функции активации (например арктангенс или cигмоида):
$$
    a[-x] = -a[x]
$$

предотвратить взрыв и затухание данных и градиентов позволяет Ксавьер инициализация:
$$
        \boldsymbol{\Omega}_k\sim N\left(0\,,\sigma^2_\Omega\right)\,,
        \qquad
        \sigma_{\Omega}^2  = \frac{2}{D_{\mathrm{in}} + D_{\mathrm{out}}}
$$

#### **2.3 Выбор метода оптимизации для обучения сети**

##### **Stochastic gradient descent (SGD)**
<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/sgd.jpg" width="500px"/>
    </figure>
</center>

Правило обновления параметров модели $\boldsymbol{\phi}_t$ для итерации $t$:

- **SGD**
$$
    \boldsymbol{\phi}_{t+1} =
    \boldsymbol{\phi}_t - \lambda \cdot \frac{\partial L_i}{\partial \boldsymbol{\phi}}
$$

- **Batch SGD**
$$
    \boldsymbol{\phi}_{t+1} =
    \boldsymbol{\phi}_t - \lambda \cdot
    \sum_{i\in{I}_t}\frac{\partial L_i}{\partial \boldsymbol{\phi}}
$$

- **GD**
$$
    \boldsymbol{\phi}_{t+1} =
    \boldsymbol{\phi}_t - \lambda
    \sum_{i\in\textbf{train data}}\frac{\partial L_i}{\partial \boldsymbol{\phi}}
$$

##### **Adaptive moment estimation (Adam)**
<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/adam.jpg" width="400px"/>
    </figure>
</center>

- **Momentum step:**
$$
    \mathbf{m}_{t+1} =
    \frac{1}{1-\beta^{t+1}}
    \left[
        \beta\cdot\mathbf{m}_t +
        (1-\beta)\cdot
        \frac{\partial L_i}{\partial \boldsymbol{\phi}}
    \right]
$$
$$
    \mathbf{v}_{t+1} =
    \frac{1}{1-\gamma^{t+1}}
    \left[
        \gamma\cdot\mathbf{v}_t +
        (1-\gamma)\cdot
        \left(\frac{\partial L_i}{\partial \boldsymbol{\phi}}\right)^2
    \right]
$$

- **Weights update step:**
$$
    \boldsymbol{\phi}_{t+1} =
    \boldsymbol{\phi}_t - \alpha \cdot
    \frac{\mathbf{m}_{t+1}}{\sqrt{\mathbf{v}_{t+1} + \varepsilon}}
$$

In [ ]:
from torch.optim import Adam

optimizer = Adam(
    model.parameters(),
    lr=1e-4,
    betas=(0.9, 0.999) # beta, gamma
)

#### **2.4 Регуляризация функции потерь**
<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/regularization.jpg" width="600px"/>
    </figure>
</center>

К функции потерь добавим дополнительное слагаемое:
$$
    L[\boldsymbol{\phi}] + \mu\cdot g[\boldsymbol{\phi}]
$$

Например, для $L_2$ регуляризации:
$$
    g[\boldsymbol{\phi}] = \sum_i \phi^2_i
$$

$L_2$ регуляризацию можно заменить **AdamW** оптимизатором, в котором $L_2$ регуляризация выполняется неявно:

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(
    model.parameters(),
    lr=1e-4,
    betas=(0.9, 0.999),
    weight_decay=0.01   # Recommended for AdamW
)

#### **2.5 Learning rate schedulers**

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/multi_step.png" width="500px"/>
    </figure>
</center>

In [ ]:
from torch.optim.lr_scheduler import MultiStepLR

scheduler = MultiStepLR(
    optimizer,
    milestones = [8, 24, 28], # эпохи, в конце которых уменьшается lr
    gamma = 0.5               # во сколько раз уменьшается lr
)

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/cosine_annealing.png" width="500px"/>
    </figure>
</center>

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR

scheduler = CosineAnnealingLR(
    optimizer,
    T_max = 32,    # Maximum number of iterations.
    eta_min = 1e-4 # Minimum learning rate.
)

#### **2.6 Ранняя остановка обучения (early stopping)**

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/early_stopping.png" width="500px"/>
    </figure>
</center>

- **Early stopping** - остановка процедуры обучения до того, как она полностью завершилась.

- Ранняя остановка может уменьшить переобучение.

#### **2.7 Аугментация изображений во время обучения**

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/aug.jpg" width="800px"/>
    </figure>
</center>

**Цели аугментации:**

- Научить модель оставаться инвариантной к несущественным, для данной задачи, трансформациям изображений.

- Генерация дополнительных обучающих данных.


### **3. Классификация изображений Fashion-MNIST**

На примере задачи классификации изображений из датасета Fashion-MNIST, рассмотрим использование некоторых из методов регуляризации.

#### **Fasion-MNIST dataset**

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/figures/02/fashion_mnist.jpg" width="700px"/>
    </figure>
</center>

- Размер изображений: 28x28 пикселей

- Цветовое пространство: grayscale

- Количество классов: 10 (одежда)

- Размер набора данных: 60 000 обучающих и 10 000 тестовых изображений


In [ ]:
# классы Fasion-MNIST
class_names = {
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot"
}

В torchvision есть класс для работы Fashion-MNIST:

https://docs.pytorch.org/vision/main/datasets.html#image-classification


Для того чтобы попрактиковаться, напишем свою реализацию класса `torch.utils.data.Dataset` для Fashion-MNIST.

Загрузим и распакуем Fashion-MNIST данные с Google Disk:

In [ ]:
from src.utils import download_and_extract

download_and_extract("1KpBql_Rpc1dtBSwFTT7UZdNLUoDde6YQ", "fashion-mnist.zip", DATA_DIR)

Создадим датасет для обучения:

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torchvision import transforms as T
from PIL import Image

class DataSet(Dataset):
    def __init__(
        self,
        files,    # список файлов изображений
        transform # аугментация изображений
    ):
        super().__init__()

        # изображения открываются, но не загружаются
        self.images = [Image.open(f) for f in files]

        # вычисляем индексы классов по имени директории
        self.labels = [int(f.parent.name) for f in files]

        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]

        x = self.transform(image)
        y = self.labels[index]

        return x, y


# список всех файлов в train датасете
files = list((DATA_DIR / "fashion-mnist/train").glob("**/*.png"))

# разделим файлы изображений на файлы для обучения и файлы для валидации
train_files, val_files = random_split(
    files,
    [50_000, 10_000],
    generator=torch.Generator().manual_seed(42)
)

# аугментация для train изображений
train_transform = T.Compose([
    # цветовая аугментация
    T.ColorJitter(
        brightness=(0.9, 1.1),
        contrast=(0.9, 1.1),
        # для grayscale изображений изменять насыщенность и тон не надо
        # saturation=(0.9, 1.1),
        # hue=0.25
    ),
    # геометрическая аугментация
    T.RandomAffine(degrees=3, translate=(0.01, 0.01)),
    # нормализация: (uint8 image) -> (float tensor) / 255
    T.ToTensor()
])

train_dataset = DataSet(train_files, train_transform)

Посмотри на изображения из датасета:

In [ ]:
import numpy as np

x, y = train_dataset[2]

print(f"x.shape: {x.shape}")
print(f"y: {y}")

image = (255 * x[0].numpy()).astype(np.uint8)
name = class_names[y]

plt.figure(figsize=(3, 3))
plt.imshow(image, cmap="gray")
plt.title(f"Class: {name}")
plt.axis("off");

Data loader генерирует батчи изображений размером

**(batch_size) x (color channels) x (image height) x (image width)**:

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1000,
    num_workers=0,
    drop_last=True,
    shuffle=True
)

# первый батч
images, labels = next(iter(train_loader))

print(f"images.shape: {images.shape}")
print(f"labels.shape: {labels.shape}")

В датасете для валидации аугментацию использовать не будем:

In [ ]:
val_transform = T.Compose([
    T.ToTensor()
])

val_dataset = DataSet(val_files, val_transform)

Загрузчик батчей валидации:

In [ ]:
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=1000,
    num_workers=0,
    drop_last=False,
    shuffle=False
)


#### **Модель классификации**

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(28 * 28, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10)
        )

        # по умолчанию в Pytorch используется He инициализация
        # так что явно инициализировать веса не обязательно
        self.apply(self.init_weights)

    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # преобразуем размер x:
        # B x 1 x 28 x 28 -> B x (28 * 28)
        x = x.view(-1, 28 * 28)

        y = self.layers(x)

        return y

model = Classifier()

print(torchinfo.summary(model, input_size=x.shape, device="cpu"))

#### **Функция потерь мультиклассовой классификации**

Функция потерь для задачи мультиклассовой классификации - средняя кросс-энтропия на батче (Cross Entropy):
$$
    \mathbf{CE} =
    -\frac{1}{|I_t|}
    \sum_{i\in I_t}\mathbf{p}_i\cdot\ln \widehat{\mathbf{p}}_i
$$
где $\mathbf{p}_i$ - one-hot вектор истинных вероятностей изображения с номером $i$:
$$
   \mathbf{p}_i = (0\,,0\,,\ldots\,,1\,,0\,,\ldots\,,0)
$$
$\widehat{\mathbf{p}}_i$ - вектор предсказанных моделью вероятностей изображения с номером $i$:
$$
    \widehat{\mathbf{p}}_i = \mathrm{softmax}[\widehat{\mathbf{y}}_i] =
    \frac{\exp[\widehat{\mathbf{y}}_i]}{\sum\limits_j \exp[\widehat{y}_{ij}]}
$$

Загрузчик данных возвращает $\left\{\mathbf{y}_i\right\}_{i\in{I}_t}$.

Модель возвращает $\left\{\widehat{\mathbf{y}}_i\right\}_{i\in{I}_t}$.

Все необходимые преобразования в вероятности происходят в классе `torch.nn.CrossEntropyLoss()`.

In [ ]:
from torch import nn

criterion = nn.CrossEntropyLoss()

#### **Метрика качества**

В качестве метрики качества возьмем среднюю величину точностей по классам (macro average):

In [ ]:
from torchmetrics.classification import MulticlassAccuracy

# вычисляет точность предсказания отдельно по каждому классу
# и в конце эпохи усредняет эти точности
accuracy = MulticlassAccuracy(num_classes=10, average="macro")

#### **Оптимизатор**

В качестве оптимизатора возьмем `torch.optim.Adam`:

In [ ]:
from torch.optim import Adam

optimizer = Adam(
    model.parameters(),
    lr=0.01,
    betas=(0.9, 0.999)
)

#### **Learning rate schedulers**

In [ ]:
from torch.optim.lr_scheduler import MultiStepLR

# максимальное число эпох
n_epochs = 20

scheduler = MultiStepLR(
    optimizer,
    milestones = [10, 15], # уменьшаем learning rate на 10 и 15 эпохе
    gamma = 0.5            # во сколько раз уменьшаем lr
)

#### **Обучение модели**

In [ ]:
from tqdm import tqdm
from livelossplot import PlotLosses
from torchmetrics.classification import MulticlassAccuracy

from src.utils import set_seed

# для воспроизводимости результатов обучения
set_seed(42)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Classifier()
model = model.to(device)

optimizer = Adam(
    model.parameters(),
    lr=0.01,
    betas=(0.9, 0.999),
)

# число эпох обучения
n_epochs = 20

scheduler = MultiStepLR(
    optimizer,
    milestones = [10, 15],
    gamma = 0.5
)

# значение лучшей метрики на валидации
best_metric = 0

plot = PlotLosses(figsize=(12, 6))

for epoch in tqdm(range(1, n_epochs + 1)):
    model.train()
    batch_loss = []

    # значение метрики качества для обучения
    accuracy = MulticlassAccuracy(num_classes=10, average="macro")

    for x, y in train_loader:
        y_hat = model(x.to(device))
        loss = criterion(y_hat, y.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_loss.append(loss.detach().cpu())
        accuracy.update(y_hat.cpu(), y)

    # средний loss на обучающих данных
    train_loss = torch.tensor(batch_loss).mean()

    # вычисление метрики качества на обучающих данных
    train_acc = accuracy.compute()

    # обновление состояния learning rate scheduler
    scheduler.step()

    model.eval()
    batch_loss = []

    # значение метрики качества для валидации
    accuracy = MulticlassAccuracy(num_classes=10, average="macro")

    for x, y in val_loader:
        with torch.no_grad():
            y_hat = model(x.to(device))

        loss = criterion(y_hat, y.to(device))

        batch_loss.append(loss.cpu())
        accuracy.update(y_hat.cpu(), y)

    # средний loss на валидации
    val_loss = torch.tensor(batch_loss).mean()

    # метрика на валидации
    val_acc = accuracy.compute()

    # сохранение лучшей модели
    if best_metric < val_acc:
        best_metric = val_acc
        torch.save(model.state_dict(), DATA_DIR / "classifier.pt")

    # текущий learning rate
    current_lr = optimizer.param_groups[0]["lr"]

    plot.update({
        "loss": train_loss,
        "val_loss": val_loss,
        "acc": train_acc,
        "val_acc": val_acc,
        "lr": current_lr
    })
    plot.send()

#### **Тестирование модели**

In [ ]:
# загрузка весов модели
state_dict = torch.load(DATA_DIR / "classifier.pt", weights_only=True)

model = Classifier()
model.load_state_dict(state_dict, strict=True)

# вычислим точность предсказания модели для каждого класса
accuracy = MulticlassAccuracy(num_classes=10, average=None)

# датасет и загрузчик тестовых данных
test_files = list((DATA_DIR / "fashion-mnist/test").glob("**/*.png"))
test_dataset = DataSet(test_files, val_transform)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=20,
    num_workers=0,
    shuffle=True
)

# инференс модели и вычисление метрики
for x, y in test_loader:
    with torch.inference_mode():
        y_hat = model(x)

    accuracy.update(y_hat, y)

test_acc = accuracy.compute()

# значения метрики качества
for label, acc in enumerate(test_acc):
    print(f"Test Accuracy of {label}: {100 * acc:.1f} %")

print(f"\nTest Accuracy (Overall): {100 * test_acc.mean():.1f} %")

Инференс модели на одном тестовом батче:

In [ ]:
x, y = next(iter(test_loader))

with torch.inference_mode():
    y_hat = model(x)

# предсказание метки классов
_, preds = torch.max(y_hat, 1)

# де-нормализация изображений
images = (255 * x).numpy().astype(np.uint8)

# построение результатов предсказания
fig = plt.figure(figsize=(20, 4))

for idx in np.arange(20):
    ax = fig.add_subplot(2, 20 // 2, idx + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap="gray")

    prediction = preds[idx].item()
    label = y[idx].item()

    color = "green" if prediction == label else "red"

    ax.set_title(f"{prediction} ({label})", color=color)